In [ ]:
!pip install yfinance
!pip install yahoofinancials

In [ ]:
import numpy as np
import pandas as pd
# import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from collections import deque
import random

import yfinance as yf
from yahoofinancials import YahooFinancials

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
dataGoogle = yf.download("GOOG", start="2016-01-01", end="2017-12-31")
dataGoogle['Date'] = dataGoogle.index
dataGoogle = dataGoogle[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']]
dataGoogle = dataGoogle.reset_index(drop=True)
dataGoogle['Date'] = pd.to_datetime(dataGoogle['Date']).apply(lambda x: x.date())
dataGoogle_train = dataGoogle
dataGoogle_train

In [ ]:
dataGoogle2 = yf.download("GOOG", start="2018-01-01", end="2018-05-31")
dataGoogle2['Date'] = dataGoogle2.index
dataGoogle2 = dataGoogle2[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']]
dataGoogle2 = dataGoogle2.reset_index(drop=True)
dataGoogle2['Date'] = pd.to_datetime(dataGoogle2['Date']).apply(lambda x: x.date())
dataGoogle_test = dataGoogle2
dataGoogle_test

In [ ]:
dataGoogle_train.to_csv('/content/drive/MyDrive/Fall 2022/Reinforcement Learning/Project/dataGoogle_train_2y.csv')
dataGoogle_test.to_csv('/content/drive/MyDrive/Fall 2022/Reinforcement Learning/Project/dataGoogle_test_1y.csv')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Fall 2022/Reinforcement Learning/Project/dataGoogle_train_2y.csv')
df = df.drop(['Unnamed: 0'], axis=1)
df.head()

In [ ]:
df_test = pd.read_csv('/content/drive/MyDrive/Fall 2022/Reinforcement Learning/Project/dataGoogle_test_1y.csv')
df_test = df_test.drop(['Unnamed: 0'], axis=1)
df_test.head()

#Agent

In [ ]:
class Agent:
    def __init__(self, state_size, window_size, trend, batch_size, test_data):
        self.state_size = state_size
        self.window_size = window_size
        self.half_window = window_size // 2
        self.trend = trend #Close_price
        self.test_data = test_data
        self.action_size = 3 #Buy, Sell, Hold
        self.batch_size = batch_size
        self.memory = deque(maxlen = 1000)
        self.inventory = []

        self.gamma = 0.95
        self.epsilon = 0.5
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.999

      # Neural Network Model
        tf.reset_default_graph()
        self.sess = tf.InteractiveSession()
        self.X = tf.placeholder(tf.float32, [None, self.state_size]) #Input
        self.Y = tf.placeholder(tf.float32, [None, self.action_size]) #Output
        feed = tf.layers.dense(self.X, 256, activation = tf.nn.relu) #Middle_layer
        self.logits = tf.layers.dense(feed, self.action_size) #Output_Forward
        self.loss = tf.reduce_mean(tf.square(self.Y - self.logits)) #Average_loss
        self.optimizer = tf.train.GradientDescentOptimizer(1e-5).minimize(self.loss) #Optimizer
        self.sess.run(tf.global_variables_initializer())

    def act(self, state):
        if random.random() <= self.epsilon:
            return random.randrange(self.action_size) #Random
        return np.argmax(
            self.sess.run(self.logits, feed_dict = {self.X: state})[0] #Greedy
        )
    
    def get_state(self, t):
        window_size = self.window_size + 1
        d = t - window_size + 1
        block = self.trend[d : t + 1] if d >= 0 else -d * [self.trend[0]] + self.trend[0 : t + 1]
        res = []
        for i in range(window_size - 1):
            res.append(block[i + 1] - block[i])
        return np.array([res])

    def replay(self, batch_size):
        mini_batch = []
        l = len(self.memory)
        for i in range(l - batch_size, l):
            mini_batch.append(self.memory[i])
        replay_size = len(mini_batch) #32
        X = np.empty((replay_size, self.state_size)) #(32, 10)
        Y = np.empty((replay_size, self.action_size)) #(32, 3)
        states = np.array([a[0][0] for a in mini_batch]) #1st_value = state
        new_states = np.array([a[3][0] for a in mini_batch]) #4th_value = next_state
        Q = self.sess.run(self.logits, feed_dict = {self.X: states}) #Hashing the states into X
        Q_new = self.sess.run(self.logits, feed_dict = {self.X: new_states}) #Hashing the new_states into X
        for i in range(len(mini_batch)):
            state, action, reward, next_state, done = mini_batch[i] # 0:state, 1:action, 2:reward, 3:next_state, 4:done
            target = Q[i] 
            target[action] = reward 
            if not done:
                target[action] += self.gamma * np.amax(Q_new[i]) #Reward calculation using Incremental_implementation
            X[i] = state
            Y[i] = target
        loss, _ = self.sess.run(
            [self.loss, self.optimizer], feed_dict = {self.X: X, self.Y: Y}
        )
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
        return loss
        
    def train(self, iterations, checkpoint, initial_money):
        lossV = []
        profitV = []
        profitIter = []
        for i in range(iterations):
            loss_temp = []
            profit_temp = []
            total_profit = 0
            inventory = []
            state = self.get_state(0)
            starting_money = initial_money
            for t in range(0, len(self.trend) - 1):
                action = self.act(state)
                next_state = self.get_state(t + 1)
                
              #Buy
                if action == 1 and starting_money >= self.trend[t] and t < (len(self.trend) - self.half_window):
                    inventory.append(self.trend[t])
                    starting_money -= self.trend[t]
                
              #Sell
                elif action == 2 and len(inventory) > 0:
                    bought_price = inventory.pop(0)
                    total_profit += self.trend[t] - bought_price
                    starting_money += self.trend[t]

               #Reward      
                reward = ((starting_money - initial_money) / initial_money)
                self.memory.append((state, action, reward, 
                                    next_state, starting_money < initial_money))
                
                state = next_state
                batch_size = min(self.batch_size, len(self.memory))
                loss = self.replay(batch_size)
                loss_temp.append(loss)
                profit_temp.append(total_profit)
            if (i+1) % checkpoint == 0:
                print('epoch: %d, total profit: %f, loss: %f, total money: %f'%(i + 1, total_profit, loss, starting_money))
                profitIter.append(total_profit)
            lossV.append(np.mean(loss_temp))
            profitV.append(np.mean(profit_temp))
        return lossV, profitV, profitIter

    def evaluate(self, initial_money):
        starting_money = initial_money
        states_sell = []
        states_buy = []
        inventory = []
        state = self.get_state(0)
        for t in range(0, len(self.test_data) - 1):
            action = self.act(state)
            next_state = self.get_state(t + 1)
            
          # Buy
            if action == 1 and initial_money >= self.test_data[t] and t < (len(self.test_data) - self.half_window):
                inventory.append(self.test_data[t])
                initial_money -= self.test_data[t]
                states_buy.append(t)
                # print('day %d: buy 1 unit at price %f, total balance %f'% (t, self.test_data[t], initial_money))
                
          # Sell 
            elif action == 2 and len(inventory):
                bought_price = inventory.pop(0)
                initial_money += self.test_data[t]
                states_sell.append(t)
                try:
                    reward = ((close[t] - bought_price) / bought_price) * 100
                except:
                    reward = 0
                # print('day %d, sell 1 unit at price %f, investment %f %%, total balance %f,'% (t, close[t], reward, initial_money))
            
            state = next_state
        reward = ((initial_money - starting_money) / starting_money) * 100
        total_gains = initial_money - starting_money
        return states_buy, states_sell, total_gains, reward

In [ ]:
close = df.Close.values.tolist()
close_test = df_test.Close.values.tolist()
initial_money = 10000
window_size = 5
batch_size = 32
agent = Agent(state_size = window_size, 
              window_size = window_size, 
              trend = close, 
              test_data = close_test,
              batch_size = batch_size)
lossV, profitV, profitIter = agent.train(iterations = 2000, checkpoint = 10, initial_money = initial_money)

In [ ]:
fig = plt.figure(figsize = (20,10))
plt.plot(lossV, color='r')
plt.title('Loss during Training')
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.show()

In [ ]:
fig = plt.figure(figsize = (20,10))
plt.plot(profitV, color='r')
plt.title('Profit during Training (Final Profit: %f)'%(profitV[len(profitV)-1]))
plt.xlabel('Iterations')
plt.ylabel('Profit')
plt.show()

In [ ]:
fig = plt.figure(figsize = (20,10))
plt.plot(profitIter, color='r')
plt.title('Profit during Training (Final Profit: %f)'%(profitIter[len(profitIter)-1]))
plt.xlabel('Iterations')
plt.ylabel('Profit')
plt.show()

#Test

In [ ]:
states_buy, states_sell, total_gains, reward = agent.evaluate(initial_money = initial_money)

In [ ]:
fig = plt.figure(figsize = (20,10))
plt.plot(close_test, color='r', lw=2.)
plt.plot(close_test, '^', markersize=10, color='m', label = 'buying signal', markevery = states_buy)
plt.plot(close_test, 'v', markersize=10, color='k', label = 'selling signal', markevery = states_sell)
plt.title('Testing Close Price (Total Profit: %f)'%(total_gains))
plt.legend()
plt.show()